In [1]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate/Fig_config_utilities.py'

<class 'numpy.ndarray'> (3187, 16101)
<class 'numpy.ndarray'> (3187, 16101)


In [2]:
# analyzing noticeable heavy tails qualities within distribution of model prediction residuals 
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
# from tail_analysis_clean import HeavyTailAnalyzer

In [8]:
# =============================================================================
# MAIN ANALYSIS CLASS
# =============================================================================

class HeavyTailAnalyzer:
    """
    Analyzes heavy-tailed residuals in gene expression predictions.
    
    Tests the hypothesis: "Most predictions are good, but a few genes have 
    catastrophically wrong predictions, creating heavy tails in the residual 
    distribution."
    """
    
    def __init__(self, residuals, gene_ids, expression_matrix, model_name="Model"):
        """
        Initialize the analyzer with residuals and gene information.
        
        Parameters
        ----------
        residuals : array-like, shape (n_predictions,)
            Pre-calculated residuals: y_true - y_pred, flattened to 1D
            
        gene_ids : array-like, shape (n_predictions,)
            Gene identifier for each prediction (same length as residuals)
            Example: ['GENE1', 'GENE1', 'GENE1', 'GENE2', 'GENE2', ...]
            
        expression_matrix : pd.DataFrame, shape (n_genes, n_samples)
            Expression values with genes as rows and samples as columns
            Used to calculate gene-level statistics (mean, variance, CV)
            
        model_name : str
            Name of the model being analyzed (for reporting)
        
        Example
        -------
        >>> residuals = y_true.ravel() - y_pred.ravel()
        >>> gene_ids = np.repeat(gene_names, n_samples)
        >>> analyzer = HeavyTailAnalyzer(residuals, gene_ids, expr_matrix, "KG-RNN")
        """
        self.residuals = np.array(residuals).ravel()
        self.gene_ids = np.array(gene_ids).ravel()
        self.model_name = model_name
        
        # Validate inputs
        if len(self.residuals) != len(self.gene_ids):
            raise ValueError(
                f"Residuals ({len(self.residuals)}) and gene_ids ({len(self.gene_ids)}) "
                "must have the same length"
            )
        
        # Store basic info
        self.n_predictions = len(self.residuals)
        self.n_unique_genes = len(np.unique(self.gene_ids))
        
        print(f"\n{'='*70}")
        print(f"INITIALIZING ANALYZER: {self.model_name}")
        print(f"{'='*70}")
        print(f"Total predictions: {self.n_predictions:,}")
        print(f"Unique genes: {self.n_unique_genes:,}")
        print(f"Mean residual: {np.mean(self.residuals):.6f}")
        print(f"Std residual: {np.std(self.residuals):.4f}")
        print(f"Kurtosis: {stats.kurtosis(self.residuals):.2f} "
              f"({'heavy tails' if stats.kurtosis(self.residuals) > 3 else 'normal'})")
        
        # Create the main dataframe
        self._build_dataframe(expression_matrix)
    
    
    def _build_dataframe(self, expression_matrix):
        """Build the internal dataframe with residuals and gene properties."""
        # Start with residuals
        self.data = pd.DataFrame({
            'gene_id': self.gene_ids,
            'residual': self.residuals,
            'abs_residual': np.abs(self.residuals)
        })
        
        # Add gene-level statistics from expression matrix
        gene_stats = self._calculate_gene_statistics(expression_matrix)
        self.data = self.data.merge(gene_stats, on='gene_id', how='left')
        
        # Create gene-level aggregates
        self.gene_summary = self._aggregate_by_gene()
    
    
    def _calculate_gene_statistics(self, expression_matrix):
        """
        Calculate mean, variance, and coefficient of variation for each gene.
        
        Returns DataFrame with columns: gene_id, mean_expr, variance, cv
        """
        print(f"\nCalculating gene-level statistics...")
        
        gene_stats = pd.DataFrame({
            'gene_id': expression_matrix.index,
            'mean_expr': expression_matrix.mean(axis=1),
            'variance': expression_matrix.var(axis=1),
            'cv': expression_matrix.std(axis=1) / (expression_matrix.mean(axis=1) + 1e-10)
        })
        
        print(f"  Mean expression range: [{gene_stats['mean_expr'].min():.2f}, "
              f"{gene_stats['mean_expr'].max():.2f}]")
        print(f"  Variance range: [{gene_stats['variance'].min():.2f}, "
              f"{gene_stats['variance'].max():.2f}]")
        
        return gene_stats
    
    
    def _aggregate_by_gene(self):
        """Aggregate residuals at the gene level."""
        gene_agg = self.data.groupby('gene_id').agg({
            'residual': ['mean', 'std'],
            'abs_residual': 'mean',
            'mean_expr': 'first',
            'variance': 'first',
            'cv': 'first'
        })
        
        # Flatten column names
        gene_agg.columns = ['_'.join(col).strip('_') for col in gene_agg.columns]
        gene_agg = gene_agg.reset_index()
        
        # Count predictions per gene
        counts = self.data.groupby('gene_id').size().reset_index(name='n_predictions')
        gene_agg = gene_agg.merge(counts, on='gene_id')
        
        return gene_agg
    
    
    # =========================================================================
    # TAIL IDENTIFICATION
    # =========================================================================
    
    def identify_heavy_tails(self, threshold_sigma=2.5):
        """
        Identify residuals in the heavy tails of the distribution.
        
        Definition: Tail residuals are those beyond ±threshold_sigma standard 
        deviations from the mean.
        
        Parameters
        ----------
        threshold_sigma : float, default=2.5
            Number of standard deviations to define tail boundary
            Common values: 2.0 (95%), 2.5 (98.8%), 3.0 (99.7%)
        
        Returns
        -------
        pd.DataFrame
            Subset of predictions in the tails, sorted by absolute residual
        """
        print(f"\n{'='*70}")
        print(f"IDENTIFYING HEAVY TAILS (±{threshold_sigma}σ)")
        print(f"{'='*70}")
        
        # Calculate tail boundaries
        mean_residual = np.mean(self.residuals)
        std_residual = np.std(self.residuals)
        lower_bound = mean_residual - threshold_sigma * std_residual
        upper_bound = mean_residual + threshold_sigma * std_residual
        
        print(f"Mean residual: {mean_residual:.6f}")
        print(f"Std residual: {std_residual:.4f}")
        print(f"Lower tail boundary: {lower_bound:.4f}")
        print(f"Upper tail boundary: {upper_bound:.4f}")
        
        # Flag tail residuals
        self.data['is_tail'] = (
            (self.data['residual'] < lower_bound) | 
            (self.data['residual'] > upper_bound)
        )
        
        # Extract tail data
        tail_data = self.data[self.data['is_tail']].copy()
        tail_data = tail_data.sort_values('abs_residual', ascending=False)
        
        # Report statistics
        n_tail = len(tail_data)
        pct_tail = 100 * n_tail / self.n_predictions
        n_genes_in_tail = tail_data['gene_id'].nunique()
        
        print(f"\nRESULTS:")
        print(f"  Tail predictions: {n_tail:,} ({pct_tail:.2f}% of total)")
        print(f"  Body predictions: {self.n_predictions - n_tail:,} "
              f"({100 - pct_tail:.2f}% of total)")
        print(f"  Unique genes affected: {n_genes_in_tail} "
              f"({100*n_genes_in_tail/self.n_unique_genes:.1f}% of genes)")
        
        # Show worst predictions
        print(f"\n  Top 10 worst predictions:")
        worst_10 = tail_data.nlargest(10, 'abs_residual')
        for idx, row in worst_10.iterrows():
            print(f"    {row['gene_id']}: residual = {row['residual']:.4f}, "
                  f"variance = {row['variance']:.2f}")
        
        self.tail_data = tail_data
        self.threshold_sigma = threshold_sigma
        
        return tail_data
    
    
    def compare_tail_vs_body(self):
        """
        Compare gene properties between tail and body residuals.
        
        Tests whether genes with catastrophic predictions (tails) have 
        different characteristics than genes with good predictions (body).
        
        Uses Mann-Whitney U test (non-parametric) to compare:
        - Mean expression level
        - Expression variance
        - Coefficient of variation
        
        Returns
        -------
        dict
            Dictionary with test results and summary statistics
        """
        if 'is_tail' not in self.data.columns:
            raise ValueError("Must run identify_heavy_tails() first")
        
        print(f"\n{'='*70}")
        print(f"COMPARING TAIL vs BODY PROPERTIES")
        print(f"{'='*70}")
        
        # Split data
        tail = self.data[self.data['is_tail']]
        body = self.data[~self.data['is_tail']]
        
        results = {}
        
        # --- Compare residual magnitudes ---
        print(f"\n1. RESIDUAL MAGNITUDES")
        print(f"   {'Tail':8s} | {'Body':8s} | Difference")
        print(f"   {'-'*8} | {'-'*8} | {'-'*10}")
        
        tail_mean_abs = tail['abs_residual'].mean()
        body_mean_abs = body['abs_residual'].mean()
        print(f"   {tail_mean_abs:8.4f} | {body_mean_abs:8.4f} | "
              f"{tail_mean_abs/body_mean_abs:.2f}x larger")
        
        results['abs_residual_tail'] = tail_mean_abs
        results['abs_residual_body'] = body_mean_abs
        
        # --- Compare gene properties ---
        properties = [
            ('mean_expr', 'Mean Expression'),
            ('variance', 'Variance'),
            ('cv', 'Coefficient of Variation')
        ]
        
        for col, label in properties:
            print(f"\n2. {label.upper()}")
            print(f"   {'Metric':20s} | {'Tail':10s} | {'Body':10s}")
            print(f"   {'-'*20} | {'-'*10} | {'-'*10}")
            
            # Summary statistics
            tail_mean = tail[col].mean()
            tail_median = tail[col].median()
            body_mean = body[col].mean()
            body_median = body[col].median()
            
            print(f"   {'Mean':20s} | {tail_mean:10.4f} | {body_mean:10.4f}")
            print(f"   {'Median':20s} | {tail_median:10.4f} | {body_median:10.4f}")
            
            # Statistical test
            tail_values = tail[col].dropna()
            body_values = body[col].dropna()
            
            if len(tail_values) > 0 and len(body_values) > 0:
                statistic, p_value = stats.mannwhitneyu(
                    tail_values, body_values, alternative='two-sided'
                )
                
                # Interpret p-value
                if p_value < 0.001:
                    sig = "***"
                    interp = "HIGHLY SIGNIFICANT"
                elif p_value < 0.01:
                    sig = "**"
                    interp = "SIGNIFICANT"
                elif p_value < 0.05:
                    sig = "*"
                    interp = "SIGNIFICANT"
                else:
                    sig = "ns"
                    interp = "NOT SIGNIFICANT"
                
                print(f"   {'Mann-Whitney U':20s} | p = {p_value:.4e} {sig}")
                print(f"   {'Interpretation':20s} | {interp}")
                
                results[f'{col}_p_value'] = p_value
                results[f'{col}_significant'] = p_value < 0.05
            else:
                print(f"   Insufficient data for statistical test")
        
        return results
    
    
    # =========================================================================
    # PERFORMANCE STRATIFICATION
    # =========================================================================
    
    def stratify_performance(self, y_true_by_gene, y_pred_by_gene, 
                            property='variance', n_bins=5):
        """
        Stratify model performance by gene property.
        
        This tests whether prediction accuracy correlates with gene characteristics.
        For example: "Do high-variance genes have worse R² scores?"
        
        Parameters
        ----------
        y_true_by_gene : dict
            Dictionary mapping gene_id -> array of true values
            Example: {'GENE1': array([5.2, 6.1, ...]), 'GENE2': array([...])}
            
        y_pred_by_gene : dict
            Dictionary mapping gene_id -> array of predicted values
            Same structure as y_true_by_gene
            
        property : str, default='variance'
            Gene property to bin by: 'variance', 'mean_expr', or 'cv'
            
        n_bins : int, default=5
            Number of quantile bins (e.g., 5 = quintiles)
        
        Returns
        -------
        pd.DataFrame
            Performance metrics for each bin
        """
        print(f"\n{'='*70}")
        print(f"STRATIFYING PERFORMANCE BY GENE {property.upper()}")
        print(f"{'='*70}")
        
        # Validate property
        valid_properties = ['variance', 'mean_expr', 'cv']
        if property not in valid_properties:
            raise ValueError(f"property must be one of {valid_properties}")
        
        # Calculate per-gene R² and metrics
        print(f"\nCalculating per-gene metrics...")
        gene_metrics = self._calculate_gene_metrics(y_true_by_gene, y_pred_by_gene)
        
        # Merge with gene summary
        analysis_df = self.gene_summary.merge(gene_metrics, on='gene_id', how='inner')
        
        # Create bins
        print(f"Creating {n_bins} quantile bins based on {property}...")
        try:
            analysis_df['bin'] = pd.qcut(
                analysis_df[property], 
                q=n_bins,
                labels=[f'Q{i+1}' for i in range(n_bins)],
                duplicates='drop'
            )
        except ValueError as e:
            print(f"  Warning: Could not create {n_bins} unique bins. Using fewer bins.")
            analysis_df['bin'] = pd.qcut(
                analysis_df[property], 
                q=n_bins,
                duplicates='drop'
            )
        
        # Aggregate by bin
        bin_summary = self._summarize_bins(analysis_df, property)
        
        # Test correlation
        self._test_correlation(analysis_df, property)
        
        # Store results
        self.stratification_results = {
            'property': property,
            'n_bins': n_bins,
            'bin_summary': bin_summary,
            'full_data': analysis_df
        }
        
        return bin_summary
    
    
    def _calculate_gene_metrics(self, y_true_by_gene, y_pred_by_gene):
        """Calculate R², MSE, and MAE for each gene."""
        metrics_list = []
        
        for gene_id in self.gene_summary['gene_id']:
            if gene_id not in y_true_by_gene or gene_id not in y_pred_by_gene:
                continue
            
            y_true = np.array(y_true_by_gene[gene_id])
            y_pred = np.array(y_pred_by_gene[gene_id])
            
            # Need at least 2 samples for R²
            if len(y_true) < 2:
                continue
            
            # Calculate metrics
            r2 = r2_score(y_true, y_pred)
            mse = mean_squared_error(y_true, y_pred)
            mae = mean_absolute_error(y_true, y_pred)
            
            metrics_list.append({
                'gene_id': gene_id,
                'r2': r2,
                'mse': mse,
                'mae': mae
            })
        
        print(f"  Calculated metrics for {len(metrics_list)} genes")
        return pd.DataFrame(metrics_list)
    
    
    def _summarize_bins(self, df, property):
        """Summarize performance metrics by bin."""
        summary = df.groupby('bin').agg({
            'r2': ['mean', 'std', 'median', 'min', 'max'],
            'mse': ['mean', 'median'],
            'mae': ['mean', 'median'],
            property: ['mean', 'min', 'max'],
            'gene_id': 'count'
        }).reset_index()
        
        # Flatten column names
        summary.columns = ['_'.join(col).strip('_') for col in summary.columns]
        summary.rename(columns={'gene_id_count': 'n_genes'}, inplace=True)
        
        # Print formatted table
        print(f"\n{'='*70}")
        print(f"PERFORMANCE BY {property.upper()} QUANTILE")
        print(f"{'='*70}\n")
        
        print(f"{'Bin':6s} | {'N Genes':8s} | {property:12s} | "
              f"{'R² Mean':8s} | {'R² Median':10s}")
        print(f"{'-'*6} | {'-'*8} | {'-'*12} | {'-'*8} | {'-'*10}")
        
        for _, row in summary.iterrows():
            bin_name = row['bin']
            n_genes = int(row['n_genes'])
            prop_mean = row[f'{property}_mean']
            r2_mean = row['r2_mean']
            r2_median = row['r2_median']
            
            print(f"{bin_name:6s} | {n_genes:8,d} | {prop_mean:12.4f} | "
                  f"{r2_mean:8.4f} | {r2_median:10.4f}")
        
        return summary
    
    
    def _test_correlation(self, df, property):
        """Test correlation between gene property and R²."""
        print(f"\n{'='*70}")
        print(f"CORRELATION: {property.upper()} vs R²")
        print(f"{'='*70}")
        
        # Remove NaN values
        valid_df = df.dropna(subset=[property, 'r2'])
        
        if len(valid_df) < 3:
            print("Insufficient data for correlation test")
            return
        
        # Spearman (rank-based, robust to outliers)
        spearman_r, spearman_p = spearmanr(valid_df[property], valid_df['r2'])
        
        # Pearson (linear correlation)
        pearson_r, pearson_p = pearsonr(valid_df[property], valid_df['r2'])
        
        print(f"\nSpearman Correlation (rank-based):")
        print(f"  ρ = {spearman_r:.4f}")
        print(f"  p-value = {spearman_p:.4e}")
        print(f"  Interpretation: {self._interpret_correlation(spearman_r, spearman_p)}")
        
        print(f"\nPearson Correlation (linear):")
        print(f"  r = {pearson_r:.4f}")
        print(f"  p-value = {pearson_p:.4e}")
        print(f"  Interpretation: {self._interpret_correlation(pearson_r, pearson_p)}")
    
    
    @staticmethod
    def _interpret_correlation(r, p):
        """Interpret correlation coefficient and p-value."""
        # Significance
        if p < 0.001:
            sig = "highly significant"
        elif p < 0.01:
            sig = "significant"
        elif p < 0.05:
            sig = "marginally significant"
        else:
            sig = "not significant"
        
        # Strength and direction
        abs_r = abs(r)
        if abs_r > 0.7:
            strength = "strong"
        elif abs_r > 0.4:
            strength = "moderate"
        elif abs_r > 0.2:
            strength = "weak"
        else:
            strength = "very weak"
        
        direction = "positive" if r > 0 else "negative"
        
        return f"{strength} {direction} correlation, {sig}"
    
    
    # =========================================================================
    # EXPORT & REPORTING
    # =========================================================================
    
    def export_results(self, output_dir='.', prefix='analysis'):
        """
        Export analysis results to CSV files.
        
        Parameters
        ----------
        output_dir : str, default='.'
            Directory to save files
        prefix : str, default='analysis'
            Prefix for output filenames
        
        Creates Files
        -------------
        {prefix}_{model}_samples.csv : All predictions with tail flags
        {prefix}_{model}_genes.csv : Gene-level summary statistics
        {prefix}_{model}_worst50.csv : Top 50 worst predictions
        """
        import os
        os.makedirs(output_dir, exist_ok=True)
        
        model_safe = self.model_name.replace(' ', '_').replace('/', '-')
        
        # 1. Sample-level data
        sample_file = os.path.join(output_dir, f'{prefix}_{model_safe}_samples.csv')
        self.data.to_csv(sample_file, index=False)
        print(f"\n✓ Saved: {sample_file}")
        print(f"  ({len(self.data):,} predictions)")
        
        # 2. Gene-level summary
        gene_file = os.path.join(output_dir, f'{prefix}_{model_safe}_genes.csv')
        self.gene_summary.to_csv(gene_file, index=False)
        print(f"✓ Saved: {gene_file}")
        print(f"  ({len(self.gene_summary):,} genes)")
        
        # 3. Worst predictions
        worst_file = os.path.join(output_dir, f'{prefix}_{model_safe}_worst50.csv')
        worst_50 = self.data.nlargest(50, 'abs_residual')
        worst_50.to_csv(worst_file, index=False)
        print(f"✓ Saved: {worst_file}")
        print(f"  (Top 50 catastrophic predictions)")
        
        print(f"\nAll results exported to: {output_dir}/")

In [4]:
### getting function inputs together 

# data loading and flattening for each model 

mlr_y_pred_unflattened = mlr_y_pred
xgbrf_y_pred_unflattened = xgbrf_y_pred

y_true = y_test_centered.ravel()
mlr_y_pred = mlr_y_pred.ravel()
xgbrf_y_pred = xgbrf_y_pred.ravel()

# generating prediction residuals for each model
mlr_residuals = y_true - mlr_y_pred
xgbrf_residuals = y_true - xgbrf_y_pred

# generating a dataframe from centered y_train with the gene ID names for better indexing 
train_expr_df = pd.DataFrame(
    y_train_centered.T,  # transpose: (n_genes, n_train_samples)
    index=y_train_gene_names  # gene names as row index
)

# generating gene_ids array by repeating gene names for each sample
# using np.tile because your data is (samples × genes)
n_test_samples = y_test_centered.shape[0]
n_genes = y_test_centered.shape[1]
gene_ids_test = np.tile(y_train_gene_names, n_test_samples)


In [16]:
# initial analysis of model predictions (both body and tails)

mlr_analyzer = HeavyTailAnalyzer(
        residuals=mlr_residuals,
        gene_ids=gene_ids_test, # gene IDs used in y training run (np.tiled to ensure they match 1D formatting of the residuals)
        expression_matrix=train_expr_df, # transposed y training set (centered) ONLY to prevent leakage 
        model_name="MLR" # mlr for this first analysis
    )

xgbrf_analyzer = HeavyTailAnalyzer(
        residuals=xgbrf_y_pred,
        gene_ids=gene_ids_test, # gene IDs used in y training run (np.tiled to ensure they match 1D formatting of the residuals)
        expression_matrix=train_expr_df, # transposed y training set (centered)
        model_name="XGBRF" # xgbrf for this second analysis
    )



INITIALIZING ANALYZER: MLR
Total predictions: 51,313,887
Unique genes: 16,101
Mean residual: -0.000276
Std residual: 0.1769
Kurtosis: 32.44 (heavy tails)

Calculating gene-level statistics...
  Mean expression range: [-0.00, 0.00]
  Variance range: [0.00, 2.84]

INITIALIZING ANALYZER: XGBRF
Total predictions: 51,313,887
Unique genes: 16,101
Mean residual: 0.001872
Std residual: 0.3973
Kurtosis: 3.56 (heavy tails)

Calculating gene-level statistics...
  Mean expression range: [-0.00, 0.00]
  Variance range: [0.00, 2.84]


In [19]:
# higher resolution inspection of tails for each model (how many there, which genes involved, worst cases)
mlr_tail_data = mlr_analyzer.identify_heavy_tails(threshold_sigma=2.5)


IDENTIFYING HEAVY TAILS (±2.5σ)
Mean residual: -0.000276
Std residual: 0.1769
Lower tail boundary: -0.4424
Upper tail boundary: 0.4419

RESULTS:
  Tail predictions: 1,422,580 (2.77% of total)
  Body predictions: 49,891,307 (97.23% of total)
  Unique genes affected: 14979 (93.0% of genes)

  Top 10 worst predictions:
    RPS4Y1: residual = -7.6462, variance = 1.06
    IGKC: residual = 6.1538, variance = 1.78
    MT1A: residual = 6.0703, variance = 0.70
    MT1G: residual = 5.7812, variance = 1.83
    RPS4Y1: residual = 5.7408, variance = 1.06
    HCST: residual = 5.7049, variance = 0.51
    S100A9: residual = 5.6819, variance = 0.95
    APOA1: residual = -5.6707, variance = 2.40
    PLA2G2A: residual = -5.4945, variance = 1.12
    IGKC: residual = 5.4040, variance = 1.78


In [ ]:
# same for xgbrf
# interesting that CSH1 and CSH2 come back, CSH1 needs to be looked at for biological significance while CSH2 might just have outliers or be functionally related to CSH1 
# differences in architecture means I have to approach this analysis differently, I need to look for 10-worst genes across ALL models not just 
xgbrf_tail_data = xgbrf_analyzer.identify_heavy_tails(threshold_sigma=2.5)


IDENTIFYING HEAVY TAILS (±2.5σ)
Mean residual: 0.001872
Std residual: 0.3973
Lower tail boundary: -0.9913
Upper tail boundary: 0.9950

RESULTS:
  Tail predictions: 1,574,022 (3.07% of total)
  Body predictions: 49,739,865 (96.93% of total)
  Unique genes affected: 14603 (90.7% of genes)

  Top 10 worst predictions:
    CSH1: residual = 5.2218, variance = 0.44
    CSH1: residual = 5.2218, variance = 0.44
    CSH1: residual = 5.2218, variance = 0.44
    CSN2: residual = 5.2117, variance = 0.03
    CSH1: residual = 5.1999, variance = 0.44
    CSH1: residual = 5.1782, variance = 0.44
    CSH1: residual = 5.1782, variance = 0.44
    CSH1: residual = 5.1782, variance = 0.44
    CSH1: residual = 5.1782, variance = 0.44
    CSH1: residual = 5.1645, variance = 0.44


In [ ]:
# further analysis of tails across all XGBRF models - which genes show up consistently? 
# roughly 20% of genes are coming back with catastrophic failure which is masked by agg R^2
# some genes with significanly poorer performan than others (ex: APCS has 84.5% of its instances in heavy tails)


# Count how many times each gene appears in tails
gene_tail_counts = xgbrf_tail_data['gene_id'].value_counts()

print(f"\n{'='*70}")
print("GENES WITH MOST TAIL PREDICTIONS (SYSTEMATIC FAILURES)")
print(f"{'='*70}\n")
print(f"{'Gene':15s} | {'Tail Count':12s} | {'% of Gene Predictions':20s}")
print(f"{'-'*15} | {'-'*12} | {'-'*20}")

n_test_samples = y_test_centered.shape[0]

for gene_id, count in gene_tail_counts.head(20).items():
    pct_of_gene = 100 * count / n_test_samples
    print(f"{gene_id:15s} | {count:12,d} | {pct_of_gene:19.2f}%")

print(f"\n{'='*70}")
print("INTERPRETATION:")
print(f"{'='*70}")
print("High % = That gene's dedicated XGBRF model is systematically failing")
print("Low % = Random outliers, gene model is generally okay")
print(f"{'='*70}\n")

# Identify genes to potentially retrain or remove
problematic_genes = gene_tail_counts[gene_tail_counts > 0.05 * n_test_samples]  # >5% failure rate
print(f"\nProblematic genes (>5% failure rate): {len(problematic_genes)}")
print(f"Consider: retraining with different hyperparameters or removing from model")


GENES WITH MOST TAIL PREDICTIONS (SYSTEMATIC FAILURES)

Gene            | Tail Count   | % of Gene Predictions
--------------- | ------------ | --------------------
APCS            |        2,693 |               84.50%
ALDOB           |        2,405 |               75.46%
GC              |        2,372 |               74.43%
C9              |        2,365 |               74.21%
CYP2C8          |        2,190 |               68.72%
LBP             |        2,171 |               68.12%
F9              |        2,164 |               67.90%
CYP3A4          |        2,156 |               67.65%
CYP2E1          |        2,121 |               66.55%
FGA             |        2,096 |               65.77%
SAA1            |        2,092 |               65.64%
SERPINC1        |        2,087 |               65.48%
PLG             |        2,052 |               64.39%
HP              |        2,050 |               64.32%
ALB             |        2,031 |               63.73%
KNG1            |       

In [ ]:
# comparing properties between body and tails
# variance is the super interesting metric in this case - the tails are of higher variance than the body (model is predicting worse on high-variance genes)
mlr_comparison = mlr_analyzer.compare_tail_vs_body()


COMPARING TAIL vs BODY PROPERTIES

1. RESIDUAL MAGNITUDES
   Tail     | Body     | Difference
   -------- | -------- | ----------
     0.7354 |   0.0798 | 9.22x larger

2. MEAN EXPRESSION
   Metric               | Tail       | Body      
   -------------------- | ---------- | ----------
   Mean                 |     0.0000 |     0.0000
   Median               |     0.0000 |    -0.0000
   Mann-Whitney U       | p = 2.0858e-05 ***
   Interpretation       | HIGHLY SIGNIFICANT

2. VARIANCE
   Metric               | Tail       | Body      
   -------------------- | ---------- | ----------
   Mean                 |     0.3073 |     0.2028
   Median               |     0.2269 |     0.1745
   Mann-Whitney U       | p = 0.0000e+00 ***
   Interpretation       | HIGHLY SIGNIFICANT

2. COEFFICIENT OF VARIATION
   Metric               | Tail       | Body      
   -------------------- | ---------- | ----------
   Mean                 | 5123021413.1973 | 4221569489.0299
   Median               | 476

In [ ]:
# same for xgbrf

# comparing both models here:
# MLR does worse at predicting genes with a higher expression variance (non-linear realtionships)
# MLR has larger absolute error compared to XGBRF (highly variance-dependent)
# Differences in tail structure present MLR as having a more gradual decline in performance vs XGBRF that either works well or doesn't (bimodality)

# VERY IMPORTANT -> data is full of non-linear relatioships based on the complexly regulated gene expression patterns
# MLR acts as an initial blanket solution, weak at capturing genes with high-variance expression (more complexity)
# XGBRF comes in explaining more of this non-linearity and works better on said high-variance expression genes
# (ex: if TF1 is above threshold but TF2 is below then...) but still fails for 20% of genes
# Introduces the need for a robust means of tackling non-linear data that can reduce variance effect, reduce net total problematic genes and 
# have different failure modes (rare expression patterns, long-range dependent expression, etc.)

xgbrf_comparison = xgbrf_analyzer.compare_tail_vs_body()


COMPARING TAIL vs BODY PROPERTIES

1. RESIDUAL MAGNITUDES
   Tail     | Body     | Difference
   -------- | -------- | ----------
     1.2906 |   0.2455 | 5.26x larger

2. MEAN EXPRESSION
   Metric               | Tail       | Body      
   -------------------- | ---------- | ----------
   Mean                 |     0.0000 |     0.0000
   Median               |     0.0000 |    -0.0000
   Mann-Whitney U       | p = 1.8827e-61 ***
   Interpretation       | HIGHLY SIGNIFICANT

2. VARIANCE
   Metric               | Tail       | Body      
   -------------------- | ---------- | ----------
   Mean                 |     0.4772 |     0.1971
   Median               |     0.3125 |     0.1729
   Mann-Whitney U       | p = 0.0000e+00 ***
   Interpretation       | HIGHLY SIGNIFICANT

2. COEFFICIENT OF VARIATION
   Metric               | Tail       | Body      
   -------------------- | ---------- | ----------
   Mean                 | 6359722968.5542 | 4179689336.4025
   Median               | 559

In [17]:
# stratifying genes based on variance 
# creating per-gene dictionaries containing model predictions were analyzing  
y_true_by_gene = {}
mlr_y_pred_by_gene = {}
xgbrf_y_pred_by_gene = {}

for i, gene in enumerate(y_train_gene_names):
    y_true_by_gene[gene] = y_test_centered[:, i]  # test samples for this gene
    mlr_y_pred_by_gene[gene] = mlr_y_pred_unflattened[:, i]
    xgbrf_y_pred_by_gene[gene] = xgbrf_y_pred_unflattened[:, i]


# binning genes on variance for each model- do higher-variance genes have a worse R^2 

mlr_bin_summary = mlr_analyzer.stratify_performance(
    y_true_by_gene=y_test_centered,
    y_pred_by_gene=mlr_y_pred_unflattened,
    property='variance',  # does variance in training set explain R^2 in test set
    n_bins=5
    )

xgbrf_bin_summary = xgbrf_analyzer.stratify_performance(
    y_true_by_gene=y_true_by_gene,
    y_pred_by_gene=xgbrf_y_pred_unflattened,
    property='variance',  
    n_bins=5
    )

    


STRATIFYING PERFORMANCE BY GENE VARIANCE

Calculating per-gene metrics...
  Calculated metrics for 0 genes


KeyError: 'gene_id'